In [ ]:
# this three below just to add them to the lambda function 
def get_arrival_date(response, i):
    if 'arrivals' in response and len(response['arrivals']) > i and 'movement' in response['arrivals'][i] and 'actualTimeLocal' in response['arrivals'][i]['movement']:
        return response['arrivals'][i]['movement']['actualTimeLocal'].split(' ')[0]
    else:
        return response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[0]

In [ ]:
def get_arrival_date(response, i):
    if 'arrivals' in response and len(response['arrivals']) > i and 'movement' in response['arrivals'][i] and 'actualTimeLocal' in response['arrivals'][i]['movement']:
        return response['arrivals'][i]['movement']['actualTimeLocal'].split(' ')[0]
    else:
        return response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[0]

In [ ]:
def get_actual_arr_local_time(response, i):
    if 'arrivals' in response and len(response['arrivals']) > i and 'movement' in response['arrivals'][i] and 'actualTimeLocal' in response['arrivals'][i]['movement']:
        return response['arrivals'][i]['movement']['actualTimeLocal'].split(' ')[1].split('+')[0]
    else:
        return response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[1].split('+')[0]

In [ ]:

def get_delay_time(response, i):
    if 'arrivals' in response and len(response['arrivals']) > i and 'movement' in response['arrivals'][i] and 'actualTimeLocal' in response['arrivals'][i]['movement']:
        return response['arrivals'][i]['movement']['actualTimeLocal'].split(' ')[1].split('+')[1]
    else:
        return response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[1].split('+')[1]

In [ ]:
# this is the complete lambda function

import pandas as pd
import requests
from datetime import datetime
import pytz
import pymysql
import sqlalchemy as db
import json
from get_actual_arr_local_time import get_actual_arr_local_time
from get_arrival_date import get_arrival_date
from get_delay_time import get_delay_time
from datetime import datetime, timedelta

def lambda_handler(event, context):
    schema="gans"
    host="wbs-project3-db.crfchknrpixs.eu-central-1.rds.amazonaws.com"
    user="kafawi"
    password="S0795735786s"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    aws_engine = db.create_engine(con, echo=True)
    aws_connection = aws_engine.connect()
    query = "select * from airports"
    airports_df = pd.read_sql(query, aws_connection)
    df = pd.DataFrame(airports_df)
        
        
    flights_data = []

    for _, row in df.iterrows():

        icao = row['icaoCode']
        iata = row['iataCode']
        airport_id = row['airport_id']
        tommorow_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tommorow_date}T11:00/{tommorow_date}T23:00"

        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

        headers = {
            "X-RapidAPI-Key": '0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276',
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        responses = requests.get(url, headers=headers, params=querystring)
        
        
        
        if responses.status_code != 200: 
            print(f"Error - Status Code: {responses.status_code} at line{_}")
            print(f"Response Content: {responses.text}")
            print('Problem with status code')
            continue
            
        response = responses.json()
        

        for i in range(len(response['arrivals'])):

            output = {
                'airport_id': airport_id,
                'arrival_date' : get_arrival_date(response,i),
                'flight_number' : response['arrivals'][i]['number'],
                'airline' : response['arrivals'][i]['airline']['name'],
                'flight_status' : response['arrivals'][i]['status'],
                
                'scheduled_arr_local_time' : response['arrivals'][i]['movement']['scheduledTimeLocal'].split(' ')[1].split('+')[0],
                
                'actual_arr_local_time' : get_actual_arr_local_time(response, i),
              
                'scheduled_arr_UTC_time' : str(pd.to_datetime(response['arrivals'][i]['movement']['scheduledTimeUtc'])).split(' ')[1].split('+')[0],
                
                'delay_time' : get_delay_time(response, i)
                
            }

            flights_data.append(output)

    flights_df = pd.DataFrame(flights_data)
    flights_df['scheduled_arr_local_time'] = pd.to_datetime(flights_df['scheduled_arr_local_time']).dt.time
    flights_df['actual_arr_local_time'] = pd.to_datetime(flights_df['actual_arr_local_time']).dt.time
    flights_df['delay_time'] = flights_df['delay_time'].apply(lambda x: datetime.strptime(x, "%M:%S").strftime("%H:%M:%S"))

    flights_df.to_sql("flights", aws_connection, if_exists="append", index=False)
    aws_connection.commit()
    aws_connection.close()
        
        
        
        
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }